In [87]:
import requests
from pprint import pprint as pp
import numpy as np
import pandas as pd

# Actions to get all the game ID urls for every season

In [81]:
lt = []
for year in range(2009,2019):
    url = "http://www.nfl.com/feeds-rs/schedules/{}.json".format(year)
    game_id_url = "http://www.nfl.com/feeds-rs/scores/byGame/"
    response = requests.get(url)
    results = response.json()
    games = results['gameSchedules']
    game_ids = []
    game_ids = ["{}{}.json".format(game_id_url, item[-1]) for game in games for item in game.items() if item[0] == "gameId"]
    lt.extend(game_ids)

In [112]:
r_lt = []
for year in range(2018,2019):
    url = "http://www.nfl.com/feeds-rs/schedules/{}.json".format(year)
    game_id_url = "http://www.nfl.com/feeds-rs/scores/byGame/"
    response = requests.get(url)
    results = response.json()
    games = results['gameSchedules']
    game_ids = []
    game_ids = ["{}{}.json".format(game_id_url, item[-1]) for game in games for item in game.items() if item[0] == "gameId"]
    r_lt.extend(game_ids)

# Actions to get info for game

columns = ['season',
         'seasonType',
         'week',
         'gameId',
         'gameKey',
         'gameDate',
         'gameTimeEastern',
         'gameTimeLocal',
         'isoTime',
         'homeTeamId',
         'visitorTeamId',
         'homeTeamAbbr',
         'visitorTeamAbbr',
         'homeDisplayName',
         'visitorDisplayName',
         'homeNickname',
         'visitorNickname',
         'gameType',
         'siteId',
         'siteCity',
         'siteFullname',
         'siteState',
         'roofType',
         'visitorTeamScore',
         'homeTeamScore']
         
         
 home_columns = ['HgamesPlayed',
 'HfirstDownsTotal',
 'HfirstDownsByRushing',
 'HfirstDownsByPassing',
 'HfirstdownsByPenalty',
 'HthirdDownAttempted',
 'HthirdDownMade',
 'HfourthDownAttempted',
 'HfourthDownMade',
 'HoffensivePlays',
 'HoffensiveYardsTotal',
 'HoffensiveAvgYardsPerPlay',
 'HrushingPlays',
 'HrushingYardsTotal',
 'HrushingAvgYardsPerPlay',
 'HrushingFumbles',
 'HrushingLong',
 'HpassingYardsTotal',
 'HpassingNetYards',
 'HpassingAttempts',
 'HpassingCompletions',
 'HpassingInterceptions',
 'HpassingAvgYardsPerPlay',
 'HpassingAvgNetYardsPerPlayWithSacks',
 'HpassingSacks',
 'HpassingSacksYardsLost',
 'HpassingPasserRating',
 'HpassingFumbles',
 'HfieldGoalAttempted',
 'HfieldGoalBlocked',
 'HfieldGoalMade',
 'HkickingXkAtt',
 'HkickingXkBlocked',
 'HkickingXkMade',
 'HextraTwoPointAtt',
 'HextraTwoPointMade',
 'HtotalExtraPointAtt',
 'HtotalExtraPointMade',
 'HtouchdownsTotal',
 'HtouchdownsRushing',
 'HtouchdownsPassing',
 'HtouchdownsPuntReturns',
 'HtouchdownsKickoffReturns',
 'HtouchdownsFumbleReturns',
 'HtouchdownsReturned',
 'HtouchdownsDefensive',
 'Hpenalties',
 'HpenaltiesYards',
 'HdefensiveFgBlocked',
 'HdefensivePuntBlocked',
 'HdefensiveXpBlocked',
 'HdefensiveForcedFumble',
 'HdefensiveInterceptions',
 'HdefensiveInterceptionsLong',
 'HdefensiveInterceptionsTds',
 'HdefensiveInterceptionsYards',
 'HdefensivePassesDefensed',
 'HdefensiveSacks',
 'HdefensiveSafeties',
 'HdefensiveAssist',
 'HdefensiveTotalTackles',
 'HdefensiveCombineTackles',
 'HfumblesTotal',
 'HfumblesLost',
 'HkickReturns',
 'HkickReturnsLong',
 'HkickReturnsTouchdowns',
 'HkickReturnsYards',
 'HkickoffTotal',
 'HkickoffInTheEndZone',
 'HkickoffTouchbacks',
 'HkickoffReturns',
 'HkickoffReturnsYards',
 'HkickingFgAttMade1To19',
 'HkickingFgAttMade20To29',
 'HkickingFgAttMade30To39',
 'HkickingFgAttMade40To49',
 'HkickingFgAttMade50plus',
 'HpuntReturns',
 'HpuntReturnsLong',
 'HpuntReturnsTouchdowns',
 'HpuntReturnsYards',
 'HpuntingAverageYards',
 'HpuntingBlocked',
 'HpuntingLong',
 'HpuntingNetAverage',
 'HpuntingNetYards',
 'HpuntingNumberReturned',
 'HpuntingPunts',
 'HpuntingPuntsInside20',
 'HpuntingTouchbacks',
 'HpuntingYards',
 'HpuntReturnsFairCatches',
 'HreceivingReceptions',
 'HreceivingTouchdowns',
 'HreceivingLong',
 'HreceivingYards',
 'HreceivingFumbles',
 'HrecevingAverageYards',
 'HredZoneAtt',
 'HredZoneSuccess',
 'HredZoneEfficiencyPct',
 'HgoalToGoAtt',
 'HgoalToGoSuccess',
 'HgoalToGoEfficiencyPct',
 'HtotalTurnovers',
 'HtotalPointsScored',
 'HtimeOfPossSeconds',
 'HavgTimeOfPossession']
 
 visitor_columns = ['VgamesPlayed',
 'VfirstDownsTotal',
 'VfirstDownsByRushing',
 'VfirstDownsByPassing',
 'VfirstdownsByPenalty',
 'VthirdDownAttempted',
 'VthirdDownMade',
 'VfourthDownAttempted',
 'VfourthDownMade',
 'VoffensivePlays',
 'VoffensiveYardsTotal',
 'VoffensiveAvgYardsPerPlay',
 'VrushingPlays',
 'VrushingYardsTotal',
 'VrushingAvgYardsPerPlay',
 'VrushingFumbles',
 'VrushingLong',
 'VpassingYardsTotal',
 'VpassingNetYards',
 'VpassingAttempts',
 'VpassingCompletions',
 'VpassingInterceptions',
 'VpassingAvgYardsPerPlay',
 'VpassingAvgNetYardsPerPlayWithSacks',
 'VpassingSacks',
 'VpassingSacksYardsLost',
 'VpassingPasserRating',
 'VpassingFumbles',
 'VfieldGoalAttempted',
 'VfieldGoalBlocked',
 'VfieldGoalMade',
 'VkickingXkAtt',
 'VkickingXkBlocked',
 'VkickingXkMade',
 'VextraTwoPointAtt',
 'VextraTwoPointMade',
 'VtotalExtraPointAtt',
 'VtotalExtraPointMade',
 'VtouchdownsTotal',
 'VtouchdownsRushing',
 'VtouchdownsPassing',
 'VtouchdownsPuntReturns',
 'VtouchdownsKickoffReturns',
 'VtouchdownsFumbleReturns',
 'VtouchdownsReturned',
 'VtouchdownsDefensive',
 'Vpenalties',
 'VpenaltiesYards',
 'VdefensiveFgBlocked',
 'VdefensivePuntBlocked',
 'VdefensiveXpBlocked',
 'VdefensiveForcedFumble',
 'VdefensiveInterceptions',
 'VdefensiveInterceptionsLong',
 'VdefensiveInterceptionsTds',
 'VdefensiveInterceptionsYards',
 'VdefensivePassesDefensed',
 'VdefensiveSacks',
 'VdefensiveSafeties',
 'VdefensiveAssist',
 'VdefensiveTotalTackles',
 'VdefensiveCombineTackles',
 'VfumblesTotal',
 'VfumblesLost',
 'VkickReturns',
 'VkickReturnsLong',
 'VkickReturnsTouchdowns',
 'VkickReturnsYards',
 'VkickoffTotal',
 'VkickoffInTheEndZone',
 'VkickoffTouchbacks',
 'VkickoffReturns',
 'VkickoffReturnsYards',
 'VkickingFgAttMade1To19',
 'VkickingFgAttMade20To29',
 'VkickingFgAttMade30To39',
 'VkickingFgAttMade40To49',
 'VkickingFgAttMade50plus',
 'VpuntReturns',
 'VpuntReturnsLong',
 'VpuntReturnsTouchdowns',
 'VpuntReturnsYards',
 'VpuntingAverageYards',
 'VpuntingBlocked',
 'VpuntingLong',
 'VpuntingNetAverage',
 'VpuntingNetYards',
 'VpuntingNumberReturned',
 'VpuntingPunts',
 'VpuntingPuntsInside20',
 'VpuntingTouchbacks',
 'VpuntingYards',
 'VpuntReturnsFairCatches',
 'VreceivingReceptions',
 'VreceivingTouchdowns',
 'VreceivingLong',
 'VreceivingYards',
 'VreceivingFumbles',
 'VrecevingAverageYards',
 'VredZoneAtt',
 'VredZoneSuccess',
 'VredZoneEfficiencyPct',
 'VgoalToGoAtt',
 'VgoalToGoSuccess',
 'VgoalToGoEfficiencyPct',
 'VtotalTurnovers',
 'VtotalPointsScored',
 'VtimeOfPossSeconds',
 'VavgTimeOfPossession']

# Might've found something...

In [54]:
def parse_data(url):
    row = []
    g_response = requests.get(url)
    g_results = g_response.json()
    for sect in g_results.items():
        if sect[0] == 'gameSchedule':
            for item in sect[-1].items():
                if (item[0] == "visitorTeam"):
                    continue
                elif (item[0] == "homeTeam"):
                    for it in item[-1].items():
                        if it[0] == "teamId":
                            team_id = it[-1]
                elif (item[0] == "gameId"):
                    game_id = item[-1]
                elif (item[0] == "site"):
                    for it in item[-1].items():
                        row.append(it[-1])
                else:
                    row.append(item[-1])
        else:
            for item in sect[-1].items():
                if (item[0] == 'homeTeamScore') | (item[0] == 'visitorTeamScore'):
                    tot = item[-1].get('pointTotal')
                    row.append(tot)
        stat_url = "http://www.nfl.com/feeds-rs/teamGameStats/{}/{}.json".format(team_id, game_id)
    s_response = requests.get(stat_url)
    s_results = s_response.json()
    home_team_stat_line = s_results['teamStatDetail']
    visitor_team_stat_line = s_results['opponentStatDetail']
    teams_stats = [home_team_stat_line, visitor_team_stat_line]
    for team in teams_stats:
        for item in team.items():
            row.append(item[-1])
    return row

In [65]:
def get_columns(url):
    row = []
    g_response = requests.get(url)
    g_results = g_response.json()
    for sect in g_results.items():
        if sect[0] == 'gameSchedule':
            for item in sect[-1].items():
                if (item[0] == "visitorTeam"):
                    continue
                elif (item[0] == "homeTeam"):
                    for it in item[-1].items():
                        if it[0] == "teamId":
                            team_id = it[-1]
                elif (item[0] == "gameId"):
                    game_id = item[-1]
                elif (item[0] == "site"):
                    for it in item[-1].items():
                        row.append(it[0])
                else:
                    row.append(item[0])
        else:
            for item in sect[-1].items():
                if (item[0] == 'homeTeamScore') | (item[0] == 'visitorTeamScore'):
                    row.append(item[0])
        stat_url = "http://www.nfl.com/feeds-rs/teamGameStats/{}/{}.json".format(team_id, game_id)
    s_response = requests.get(stat_url)
    s_results = s_response.json()
    home_team_stat_line = s_results['teamStatDetail']
    visitor_team_stat_line = s_results['opponentStatDetail']
    teams_stats = [home_team_stat_line, visitor_team_stat_line]
    for team in teams_stats:
        for item in team.items():
            row.append(item[0])
    return row

In [113]:
rest_of_data = []
counter = 0
for l in r_lt[1:]:
    counter += 1
    row = parse_data(l)
    rest_of_data.append(row)  
    print("{} of {} complete...".format(counter, len(r_lt)))

1 of 333 complete...
2 of 333 complete...
3 of 333 complete...
4 of 333 complete...
5 of 333 complete...
6 of 333 complete...
7 of 333 complete...
8 of 333 complete...
9 of 333 complete...
10 of 333 complete...
11 of 333 complete...
12 of 333 complete...
13 of 333 complete...
14 of 333 complete...
15 of 333 complete...
16 of 333 complete...
17 of 333 complete...
18 of 333 complete...
19 of 333 complete...
20 of 333 complete...
21 of 333 complete...
22 of 333 complete...
23 of 333 complete...
24 of 333 complete...
25 of 333 complete...
26 of 333 complete...
27 of 333 complete...
28 of 333 complete...
29 of 333 complete...
30 of 333 complete...
31 of 333 complete...
32 of 333 complete...
33 of 333 complete...
34 of 333 complete...
35 of 333 complete...
36 of 333 complete...
37 of 333 complete...
38 of 333 complete...
39 of 333 complete...
40 of 333 complete...
41 of 333 complete...
42 of 333 complete...
43 of 333 complete...
44 of 333 complete...
45 of 333 complete...
46 of 333 complete.

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [114]:
df4 = pd.DataFrame(rest_of_data)
df4.columns = all_cols

In [115]:
df4.to_csv("../data/raw/game_data(2018).csv", index=False)

In [107]:
rest_of_data = []
counter = 0
for l in r_lt[1:]:
    counter += 1
    row = parse_data(l)
    rest_of_data.append(row)  
    print("{} of {} complete...".format(counter, len(r_lt)))

1 of 665 complete...
2 of 665 complete...
3 of 665 complete...
4 of 665 complete...
5 of 665 complete...
6 of 665 complete...
7 of 665 complete...
8 of 665 complete...
9 of 665 complete...
10 of 665 complete...
11 of 665 complete...
12 of 665 complete...
13 of 665 complete...
14 of 665 complete...
15 of 665 complete...
16 of 665 complete...
17 of 665 complete...
18 of 665 complete...
19 of 665 complete...
20 of 665 complete...
21 of 665 complete...
22 of 665 complete...
23 of 665 complete...
24 of 665 complete...
25 of 665 complete...
26 of 665 complete...
27 of 665 complete...
28 of 665 complete...
29 of 665 complete...
30 of 665 complete...
31 of 665 complete...
32 of 665 complete...
33 of 665 complete...
34 of 665 complete...
35 of 665 complete...
36 of 665 complete...
37 of 665 complete...
38 of 665 complete...
39 of 665 complete...
40 of 665 complete...
41 of 665 complete...
42 of 665 complete...
43 of 665 complete...
44 of 665 complete...
45 of 665 complete...
46 of 665 complete.

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [110]:
df3 = pd.DataFrame(rest_of_data)
df3.columns = all_cols

In [111]:
df3.to_csv("../data/raw/game_data(2017).csv", index=False)

In [100]:
rest_of_data = []
counter = 0
for l in r_lt[1:]:
    counter += 1
    row = parse_data(l)
    rest_of_data.append(row)  
    print("{} of 997 complete...".format(counter))

1 of 997 complete...
2 of 997 complete...
3 of 997 complete...
4 of 997 complete...
5 of 997 complete...
6 of 997 complete...
7 of 997 complete...
8 of 997 complete...
9 of 997 complete...
10 of 997 complete...
11 of 997 complete...
12 of 997 complete...
13 of 997 complete...
14 of 997 complete...
15 of 997 complete...
16 of 997 complete...
17 of 997 complete...
18 of 997 complete...
19 of 997 complete...
20 of 997 complete...
21 of 997 complete...
22 of 997 complete...
23 of 997 complete...
24 of 997 complete...
25 of 997 complete...
26 of 997 complete...
27 of 997 complete...
28 of 997 complete...
29 of 997 complete...
30 of 997 complete...
31 of 997 complete...
32 of 997 complete...
33 of 997 complete...
34 of 997 complete...
35 of 997 complete...
36 of 997 complete...
37 of 997 complete...
38 of 997 complete...
39 of 997 complete...
40 of 997 complete...
41 of 997 complete...
42 of 997 complete...
43 of 997 complete...
44 of 997 complete...
45 of 997 complete...
46 of 997 complete.

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [101]:
df2 = pd.DataFrame(rest_of_data)

In [102]:
df2.columns = all_cols

In [104]:
df2.to_csv("../data/raw/game_data(2016).csv", index=False)

In [103]:
df2.tail()

,season,seasonType,week,gameKey,gameDate,gameTimeEastern,gameTimeLocal,isoTime,homeTeamId,visitorTeamId,...,VredZoneAtt,VredZoneSuccess,VredZoneEfficiencyPct,VgoalToGoAtt,VgoalToGoSuccess,VgoalToGoEfficiencyPct,VtotalTurnovers,VtotalPointsScored,VtimeOfPossSeconds,VavgTimeOfPossession
325,2016,POST,19,57162,01/14/2017,20:15:00,20:15:00,1484442900000,3200,2120,...,None,None,None,None,None,None,3,16,1950,32:30
326,2016,POST,19,57164,01/15/2017,16:40:00,15:40:00,1484516400000,1200,1800,...,None,None,None,None,None,None,1,34,1786,29:46
327,2016,POST,19,57163,01/15/2017,20:20:00,19:20:00,1484529600000,2310,3900,...,None,None,None,None,None,None,1,18,2053,34:13
328,2016,POST,20,57165,01/22/2017,15:05:00,15:05:00,1485115500000,0200,1800,...,None,None,None,None,None,None,2,21,1581,26:21
329,2016,POST,20,57166,01/22/2017,18:40:00,18:40:00,1485128400000,3200,3900,...,None,None,None,None,None,None,2,17,1714,28:34


In [83]:
all_data = []
for l in lt:
    row = parse_data(l)
    all_data.append(row)  

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [85]:
len(lt)

3328

In [89]:
df = pd.DataFrame(all_data)

In [94]:
df.tail(50)

,season,seasonType,week,gameKey,gameDate,gameTimeEastern,gameTimeLocal,isoTime,homeTeamId,visitorTeamId,...,VredZoneAtt,VredZoneSuccess,VredZoneEfficiencyPct,VgoalToGoAtt,VgoalToGoSuccess,VgoalToGoEfficiencyPct,VtotalTurnovers,VtotalPointsScored,VtimeOfPossSeconds,VavgTimeOfPossession
2280,2015,REG,15,56722,12/20/2015,16:05:00,13:05:00,1450645500000,4600,1050,...,None,None,None,None,None,None,1,13,1527,25:27
2281,2015,REG,15,56724,12/20/2015,16:25:00,13:25:00,1450646700000,4400,2700,...,None,None,None,None,None,None,1,14,1267,21:07
2282,2015,REG,15,56725,12/20/2015,16:25:00,13:25:00,1450646700000,4500,0920,...,None,None,None,None,None,None,1,24,1859,30:59
2283,2015,REG,15,56723,12/20/2015,16:25:00,16:25:00,1450646700000,3900,1400,...,None,None,None,None,None,None,2,27,1724,28:44
2284,2015,REG,15,56719,12/20/2015,20:30:00,20:30:00,1450661400000,3700,3800,...,None,None,None,None,None,None,0,40,2248,37:28
2285,2015,REG,15,56726,12/21/2015,20:30:00,19:30:00,1450747800000,3300,1540,...,None,None,None,None,None,None,1,35,1776,29:36
2286,2015,REG,16,56727,12/24/2015,20:25:00,17:25:00,1451006700000,2520,4400,...,None,None,None,None,None,None,1,20,2104,35:04
2287,2015,REG,16,56728,12/26/2015,20:25:00,20:25:00,1451179500000,3700,5110,...,None,None,None,None,None,None,0,38,2055,34:15
2288,2015,REG,16,56741,12/27/2015,13:00:00,13:00:00,1451239200000,0325,3900,...,None,None,None,None,None,None,3,17,1540,25:40
2289,2015,REG,16,56730,12/27/2015,13:00:00,13:00:00,1451239200000,0610,1200,...,None,None,None,None,None,None,2,6,1710,28:30


In [91]:
df.columns = all_cols

In [92]:
df.head()

,season,seasonType,week,gameKey,gameDate,gameTimeEastern,gameTimeLocal,isoTime,homeTeamId,visitorTeamId,...,VredZoneAtt,VredZoneSuccess,VredZoneEfficiencyPct,VgoalToGoAtt,VgoalToGoSuccess,VgoalToGoEfficiencyPct,VtotalTurnovers,VtotalPointsScored,VtimeOfPossSeconds,VavgTimeOfPossession
0,2009,PRE,0,54723,08/09/2009,20:00:00,20:00:00,1249862400000,2100,0610,...,None,None,None,None,None,None,3,18,1622,27:02
1,2009,PRE,1,54725,08/13/2009,19:30:00,19:30:00,1250206200000,3700,3200,...,None,None,None,None,None,None,1,27,1563,26:03
2,2009,PRE,1,54726,08/13/2009,19:30:00,19:30:00,1250206200000,0325,5110,...,None,None,None,None,None,None,2,0,1456,24:16
3,2009,PRE,1,54724,08/13/2009,20:00:00,20:00:00,1250208000000,3900,3800,...,None,None,None,None,None,None,2,10,1756,29:16
4,2009,PRE,1,54727,08/13/2009,22:00:00,19:00:00,1250215200000,2520,1200,...,None,None,None,None,None,None,2,10,1768,29:28


In [95]:
df.to_csv("../data/raw/game_data(2009-2015).csv", index=False)

In [55]:
y1 = lt[0]
test = [y1[0], y1[1]]
data = []
for t in test:
    row = parse_data(t)
    data.append(row)

In [42]:
visitor_columns = []
for col in stat_columns:
    visitor_columns.append("V{}".format(col))

In [78]:
pp(all_cols)

['season',
 'seasonType',
 'week',
 'gameKey',
 'gameDate',
 'gameTimeEastern',
 'gameTimeLocal',
 'isoTime',
 'homeTeamId',
 'visitorTeamId',
 'homeTeamAbbr',
 'visitorTeamAbbr',
 'homeDisplayName',
 'visitorDisplayName',
 'homeNickname',
 'visitorNickname',
 'gameType',
 'siteId',
 'siteCity',
 'siteFullname',
 'siteState',
 'roofType',
 'visitorTeamScore',
 'homeTeamScore',
 'HgamesPlayed',
 'HfirstDownsTotal',
 'HfirstDownsByRushing',
 'HfirstDownsByPassing',
 'HfirstdownsByPenalty',
 'HthirdDownAttempted',
 'HthirdDownMade',
 'HfourthDownAttempted',
 'HfourthDownMade',
 'HoffensivePlays',
 'HoffensiveYardsTotal',
 'HoffensiveAvgYardsPerPlay',
 'HrushingPlays',
 'HrushingYardsTotal',
 'HrushingAvgYardsPerPlay',
 'HrushingFumbles',
 'HrushingLong',
 'HpassingYardsTotal',
 'HpassingNetYards',
 'HpassingAttempts',
 'HpassingCompletions',
 'HpassingInterceptions',
 'HpassingAvgYardsPerPlay',
 'HpassingAvgNetYardsPerPlayWithSacks',
 'HpassingSacks',
 'HpassingSacksYardsLost',
 'Hpassing